# Line Sampling


## TODO

```{todo}
Write this section.
```

## Syntax
```{eval-rst}
.. function:: flx.perform_Line_Sampling

    Syntax:
        ``flx.perform_Line_Sampling(LSF, U_STAR, SAMPLER, CONFIG)``

    Description:
        Perform a line sampling (structural reliability analysis).
        
    :param LSF: The limit-state function (that depends on the input random variables of sampler ``SAMPLER``).
    :type LSF: :type:`flxPara`
    :param U_STAR: The base-vector for performing the line sampling. Ideally, this point is the design point of the structural reliability problem.
    :type U_STAR: numpy.ndarray
    :param SAMPLER: A sampler for a set (or multiple sets) of random variables.
    :type SAMPLER: :class:`flx.sampler`
    :param CONFIG: 
        A configuration dictionary. The following keys are allowed in ``CONFIG``:
    
         - ``n_ls`` (*int*, default: 10): The total number of line searches to perform. Value must be positive.
         - ``ls_max_iter`` (*int*, default: 10): The maximum number of iterations per line-search. Value must be positive.
         - ``ls_tol`` (*float*, default: 0.001): Tolerance parameter for performing the line search. Value must be positive.
         - ``use_bisec`` (*bool*, default: *False*): Specifies the line-search algorithm. ``True``: bisection method; ``False``: regula falsi / secant method
         - ``extended_ls`` (*bool*, default: *False*): If ``True``, the algorithm searches for multiple roots per line-search.
         - ``verboseLog`` (*bool*, default: *False*): Verbose logging.
         - ``show_progress`` (*bool*, default: *True*): Show progress bar.
         - ``data_box`` (type :class:`flx.dataBox`): A :class:`flx.dataBox` to store/post-process all evaluations of the limit-state function performed by the line sampling algorithm.

    :type CONFIG: *dict*
    
    :returns:     
      A Python *dict* with the following information is returned:

        - ``pf`` (*float*): The probability of failure estimated by line-sampling.
        - ``N_lsf_calls`` (*int*): The total number of times the limit-state function was evaluated.
        - ``calls_per_line_search`` (*int*): The average number of limit-state function calls per line-search.
        - ``n_ls`` (*int*): The total number of line searches performed.
        
    :rtype: *dict*

```

## Application Example
(content:reliability_analysis:LS:example)=

In [1]:
import fesslix as flx
flx.load_engine()

Random Number Generator: MT19937 - initialized with rand()=396754188;
Random Number Generator: MT19937 - initialized with 1000 initial calls.


0

In [2]:
## ==============================================
## Generate input model
## ==============================================
config_rv_R = { 'name':'R', 'type':'logn', 'mu':6., 'sd':1. }
config_rv_S = { 'name':'S', 'type':'normal', 'mu':1., 'sd':1. }
rv_set = flx.rv_set( {'name':'rv_set', 'allow_x2y':True}, [ config_rv_R, config_rv_S ] )
sampler = flx.sampler(['rv_set'])

In [3]:
## ==============================================
## Limit-state function
## ==============================================
lsf_expr = "rbrv(rv_set::R)-rbrv(rv_set::S)"

In [4]:
## ==============================================
## Set up dataBox
## ==============================================
dBox = flx.dataBox(2,1)
pp_rel = dBox.register_post_processor({ 'type':'counter' })

In [5]:
## ==============================================
## FORM solution
## ==============================================
config = { 'verboseLog':True, 'opt_method':'HLRF', 'fdstep':5e-2, 'data_box':dBox }
form_res = flx.perform_FORM(lsf_expr, sampler, config)
print(form_res)
print(pp_rel.eval())

{'FORM_alpha': array([-0.5658187 ,  0.82452968]), 'FORM_beta': 3.8105978475448645, 'FORM_pf': 6.931556382892316e-05, 'pf_upper_bound': '7.028775e-4', 'dzdy': array([ 0.13952447, -0.20331966]), 'FORM_x': array([4.14195278, 4.14195104]), 'FORM_u': array([-2.15610752,  3.14195104]), 'N_lsf_calls': 16, 'N_iter': 5}
{'N': 6}


In [6]:
## ==============================================
## Line sampling
## ==============================================
config = { 'n_ls':10000, 'extended_ls':True, 'use_bisec':False, 'data_box':dBox }
ra_ls_res = flx.perform_Line_Sampling(lsf_expr, form_res['FORM_u'], sampler, config)
print(ra_ls_res)
print(pp_rel.eval())

{'pf': 6.189847873928923e-05, 'N_lsf_calls': 60004, 'calls_per_line_search': 6.0, 'n_ls': 10000}
{'N': 60010}
